In [4]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.io import arff
import data_processing as dp

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [7]:
data = arff.loadarff("../data/3year.arff")
df = pd.DataFrame(data[0])
X_train, X_test, y_train, y_test = dp.get_train_test(df)

In [11]:
X_train.info

<bound method DataFrame.info of           Attr5     Attr9    Attr15    Attr21    Attr24    Attr26    Attr27  \
1881   0.945558  0.003337  0.184894  0.000098  0.256337  0.024715  0.065267   
10000  0.945551  0.003800  0.184923  0.000074  0.256201  0.024212  0.065274   
1082   0.945557  0.002938  0.185166  0.000062  0.253662  0.024146  0.065266   
1428   0.945546  0.008662  0.184716  0.000077  0.251691  0.024110       NaN   
2251   0.945566  0.002613  0.184913  0.000071  0.254623  0.024253  0.065271   
...         ...       ...       ...       ...       ...       ...       ...   
8168   0.945544  0.002985  0.185278  0.000072  0.251971  0.024140  0.065267   
3500   0.945554  0.002738  0.184921  0.000069  0.253087  0.024211  0.065267   
7048   0.945547  0.003002  0.185272  0.000091  0.252471  0.024139  0.065267   
779    0.945559  0.003321  0.184904       NaN  0.252940  0.024366  0.065267   
9707   0.945450  0.002100  0.185004  0.000071  0.252536  0.024155  0.065267   

         Attr29    

In [9]:
model = LogisticRegression(solver='liblinear', random_state=0)
model.fit(X_train, y_train)

ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [5]:
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

NameError: name 'model' is not defined